In [48]:
import numpy as np
import pandas as pd
import os
import math
import random
import csv
from gurobi import *
#import solveengine
#
#print(solveengine.MIPModel("hhh"))

In [2]:
m = Model("Quiqup")

# Travel Matrix 

In [3]:
path = 'youri_travel_matrix_2.0.csv'
df = pd.read_csv(path)

df.head()

,id,from_loc_id,from_lon,from_lat,pickup_loc_id,pickup_lon,pickup_lat,drop_loc_id,offset_drop_lon,offset_drop_lat,MoT,distance_start_pickup,duration_start_pickup,distance_pickup_drop,duration_pickup_drop
0,1,4584882,-0.146036,51.522727,2173892,-0.182489,51.494415,3687551,-0.177908,51.490475,scooter,5342.6,509.3,614.8,60.5
1,2,1183446000,-0.245977,51.536949,4474343,-0.179382,51.416593,4585124,-0.171886,51.428223,scooter,17922.8,1680.5,1756.3,170.1
2,3,1183446000,-0.245977,51.536949,4474349,-0.206004,51.420597,4585234,-0.207966,51.406167,car,18935.6,2022.5,2406.9,292.1
3,4,1183446000,-0.245977,51.536949,4474382,-0.214428,51.555711,4586638,-0.240320,51.559694,bicycle,3800.1,959.4,2337.3,714.5
4,5,1183446000,-0.245977,51.536949,4474386,-0.124112,51.378221,4588375,-0.159019,51.361374,scooter,24190.2,2272.6,5829.1,561.0


### Lists from Travel Matrix

In [4]:
id = df.id
start_id = df.from_loc_id
start_lon = df.from_lon
start_lat = df.from_lat
pickup_id = df.pickup_loc_id
pickup_lon = df.pickup_lon
pickup_lat = df.pickup_lat
delivery_id = df.drop_loc_id
delivery_lon = df.offset_drop_lon
delivery_lat = df.offset_drop_lat
distance_start_pickup = df.distance_start_pickup
distance_pickup_delivery = df.distance_pickup_drop
time_start_pickup = df.duration_start_pickup
time_pickup_delivery = df.duration_pickup_drop
MoT = df.MoT

### Jobs information

In [6]:
#all information about the job
R = range(len(id))
a = []
routes = {}

for i in R:
    #all info about routes i
    a.append((start_id[i], start_lon[i], start_lat[i], pickup_id[i], pickup_lon[i], pickup_lat[i], delivery_id[i], delivery_lon[i], delivery_lat[i], MoT[i], time_start_pickup[i], time_pickup_delivery[i]))
    routes[id[i]] = a[i]

Insert job_id and get all the information about the job

In [62]:
route = {}
for i in R:
    route[i] = [routes[id[i]]]

In [63]:
route[0]

[(4584882,
  -0.146036267,
  51.522727289999999,
  2173892,
  -0.1824894,
  51.4944153,
  3687551,
  -0.177908444,
  51.490475140000001,
  'scooter',
  509.30000000000001,
  60.5)]

In [9]:
#distances
d1 = []
d2 = []
dist_start_pickup = {}
dist_pickup_delivery = {}
total_distance = {}

for i in R:
    #distance from start to pickup
    d1.append((distance_start_pickup[i]))
    dist_start_pickup[id[i]] = d1[i]
    
    #distance from pickup to delivery
    d2.append((distance_pickup_delivery[i]))
    dist_pickup_delivery[id[i]] = d2[i]
    
    #total distance
    total_distance[id[i]] = d1[i] + d2[i]

Insert job_id to get distance

In [10]:
dist_start_pickup[id[0]]

5342.6000000000004

In [11]:
dist_pickup_delivery[id[0]]

614.79999999999995

In [12]:
total_distance[id[0]]

5957.4000000000005

In [13]:
#time
t1 = []
t2 = []
t1_start_pickup = {}
t2_pickup_delivery = {}
total_time = {}

for i in R:
    #time from start to pickup
    t1.append((time_start_pickup[i]))
    t1_start_pickup[id[i]] = t1[i]
    
    #time from pickup to delivery
    t2.append((time_pickup_delivery[i]))
    t2_pickup_delivery[id[i]] = t2[i]
    
    #total time
    total_time[id[i]] = t1[i] + t2[i]

Insert id and get time to complete the job

In [14]:
t1_start_pickup[id[0]]

509.30000000000001

In [15]:
t2_pickup_delivery[id[0]]

60.5

In [16]:
total_time[id[0]]

569.79999999999995

# Shifts data
##### Additional information
1. There are 242 drivers for 347 shifts: Some drivers are taking breaks and make themselves available with the same shift_id
2. 38 distinct starting points
3. shifts_starting_loc_id's all have 000 at the end
4. Default shift time: 4 hours (but can be prolonged to finish a job) but data is inconsistent

In [17]:
path1 = 'Shiftsdataforselectedday_2.0.csv'
df1 = pd.read_csv(path1)

df1.head()

,disg_shift_id,disg_courier_id,transport,shift_start_time,shift_end_time,starting_loc_id,starting_lon,starting_lat
0,1,139,Scooter,10/06/2019 12:00,10/06/2019 22:00,1.178801e+09,-0.076223,51.524128
1,2,150,Scooter,10/06/2019 12:00,10/06/2019 22:00,1.178771e+09,-0.123455,51.515900
2,3,151,Scooter,10/06/2019 12:05,10/06/2019 22:05,1.178801e+09,-0.076223,51.524128
3,4,163,Scooter,10/06/2019 15:00,10/06/2019 18:06,1.178776e+09,-0.163983,51.475052
4,5,175,Scooter,10/06/2019 12:00,10/06/2019 22:15,1.178776e+09,-0.163983,51.475052


### Lists from shifts 

In [18]:
shift_id = df1.disg_shift_id
driver_id = df1.disg_courier_id
transport = df1.transport
shift_start = df1.shift_start_time
shift_end = df1.shift_end_time
driver_start_id = df1.starting_loc_id
driver_start_lon = df1.starting_lon
driver_start_lat = df1.starting_lat

### Driver information

In [66]:
D = range(len(set(driver_id)))
b = []
driver_info = {}

for i in D:
    b.append((shift_id[i], transport[i], shift_start[i], shift_end[i], driver_start_id[i], driver_start_lon[i], driver_start_lat[i]))
    driver_info[driver_id[i]] = b[i]

In [67]:
driver = {}
for i in D:
    driver[i] = [driver_info[driver_id[i]]]

Insert driver_id and get information about the driver

In [68]:
driver[1]

[(2,
  'Scooter',
  '10/06/2019 12:00',
  '10/06/2019 22:00',
  1178771000.0,
  -0.123455,
  51.515900000000002)]

In [76]:
for i in range(len(shift_id)):
    print(b[0][2])
    break

10/06/2019 12:00


In [81]:
mot = []
driver_MoT = {}

for i in D:
    mot.append((transport[i]))
    driver_MoT[driver_id[i]] = mot[i]

Insert driver_id and get MoT

In [82]:
mot[0]

'Scooter'

# Orders dataset (real data)

In [29]:
path2 = 'Jobsdataforselectedday_2.0.csv'
df2 = pd.read_csv(path2)

df2.head()

,encr_job_id,job_state,job_kind,b2b_delivery_restrictions,started_at,assigned_at,items_ready_at,pickup_arrived_at,drop_started_at,drop_arrived_at,...,cancelled_at,disg_courier_id,transport_mode,encr_pickup_id,pickup_loc_id,pickup_lat,pickup_lon,drop_loc_id,offset_drop_lat,offset_drop_lon
0,41e715a1a8401966d459e92e502018e1,complete,B2B scheduled,NaN,2019-06-10 08:00:00,2019-06-10 06:50:03.174984,NaN,2019-06-10 07:53:36.791219,2019-06-10 08:46:51.78763,2019-06-10 08:52:20.677112,...,NaN,112,Car,eb19fae70183e190ed302cfc384346ae,4582299,51.485816,-0.180878,4570506,51.493379,-0.175047
1,047b08657758ac8c0131158145e3ebaf,complete,B2C on demand,NaN,2019-06-10 08:05:24.543076,2019-06-10 08:06:03.07823,NaN,2019-06-10 08:15:18.534345,2019-06-10 08:27:24.719003,2019-06-10 08:34:07.464798,...,NaN,14,Scooter,6a7afe87251855e7b2939e87d6934034,4100353,51.515897,-0.204797,4584831,51.522585,-0.165872
2,6e398ce770b25f4383af626bd0d66d02,complete,B2C on demand,NaN,2019-06-10 08:06:52.259856,2019-06-10 08:11:40.006109,NaN,2019-06-10 08:27:54.950745,2019-06-10 08:38:20.176782,2019-06-10 08:39:18.3304,...,NaN,148,Scooter,d5651d9abc4a757b40ac3dcf70a837e0,407304,51.535844,-0.205274,3417133,51.534937,-0.211342
3,dd2e3b82ea1b32cee629b13620e92fdc,complete,B2C on demand,NaN,2019-06-10 08:17:51.181189,2019-06-10 08:19:03.450223,NaN,2019-06-10 08:19:27.978836,2019-06-10 08:33:56.746714,2019-06-10 08:35:42.456851,...,NaN,40,Bicycle,939f71635ccbb759c05ffd8dea59fee6,411098,51.489849,-0.162822,3963268,51.491147,-0.170908
4,4c672bfed253e6199321f50ff7b87090,complete,B2C on demand,NaN,2019-06-10 08:20:21.672239,2019-06-10 08:22:03.559907,NaN,2019-06-10 08:35:25.820282,2019-06-10 08:50:08.608135,2019-06-10 08:55:09.564466,...,NaN,13,Scooter,0ceb8f47864a220f07157cff0ed18b9a,658821,51.521655,-0.134368,4088718,51.531753,-0.119836


### Lists from orders dataset
The following lists are relevant for the model. This is data that can be used to assign orders to the model.

In [30]:
job_id = df2.encr_job_id
job_kind = df2.job_kind
b2b_delivery_restrictions = df2.b2b_delivery_restrictions
started_at = df2.started_at #started at is the time at which Quiqup's driver started its journey. It's not data that I need
assigned_at = df2.assigned_at
pickup_id_job = df2.pickup_loc_id

In [31]:
J = range(len(job_id))

In [32]:
for j in J:
    print(pickup_id_job[34])
    break

2173488


### Orders information 

Here I am assuming that the starting time of an job equals the time at which the job has been assigned to the system

In [33]:
start_times = []
start_time = {}

for i in range(0, 1087):
    start_times.append((assigned_at[i]))
    start_time[job_id[i]] = start_times[i]

Insert job_id and retrieve starting time of the job

In [34]:
start_time[job_id[0]]

'2019-06-10 06:50:03.174984'

### Arrival time function

There are two issues:
1. Job starting time is a list that comes from the Orders dataset. Total time to complete a job is a list that comes from the Travel Matrix. How to you deal with this situation?
2. We need to match the right 'i' with the right 'j' (i.e. route 'i' needs to correspond to the route of job 'j'). We need to find a way to link relevant i-j. 
    - Pickup ID is the same in both dataset. Is there something to do with it?
    
    
Instead of using ranges --> call i in pickup_id and use i=j 

In [35]:
arrival_time = {}

for d in D:
    for j in J:
        if pickup_id[i] == pickup_id_job[j]:
            arrival_time[i,j] = m.addConstr(started_at[j] + total_time[i], name = 'Arrival_time')
            print('Arrival_time')
        else:
            break

In [36]:
for d in D:
    for j in J:
        print(arrival_time[i,j])

KeyError: (1086, 0)

### Parameters

In [37]:
#constants
pickup_and_delivery_timeout_bike = 20
SLA_start_to_pickup = 15 #time only taken into account while driver is on the move
SLA_pickup_to_delivery = 15 #it does not take into account waiting time at pickup of driver
SLA_satisfy = 0.95 #90%-95% of SLAs must be satisfied --> 0.05 instead of 0.95?
gm_raduis = 5000 #5000m radius search to allocate driver to job

#lists
driver_status = ['busy', 'avail']
#job_kind = ['b2b_scheduled', 'b2b_demand', 'b2c_demand']
#b2b_delivery_restrictions = ['SLA', 'None']
job_state = ['complete', 'cancelled']

# Model formulation 

We have to create a function which aims to match a driver whenever a job is assigned to the model. This function needs to take into account drivers that are available and within the 5k GM, and also the drivers that are on a job, whose final destination is in the 5k GM. The function will have to compute the arrival time of the latter and calculate the time at which he would finish the next job. 
    * create a general function to match driver with job
        - MoT, arrival time, starting location, delivery location
        - then update the location of the driver that is matched to the job (i.e. the delivery location of this job 
        becomes the starting location of the driver’s next job)

Notes on the cell below:
1. Need to fix the KeyError
2. Have to create a set of var[d] or to fix it as follows: driver[d] * x(binary) = 1 if driver is assigned to job
3. Must create arrival time function to take into account the drivers on another job

In [52]:
#drivers = {}
for d in D:
    vars[d] = m.addVar(vtype=GRB.BINARY, name='drivers')

In [56]:
max_driver = m.addConstr(quicksum(vars[d] for d in D) <= 1)

KeyError: 0

In [85]:
#if distance from start to pickup is larger than 5k --> driver 'd' can't be assigned to job
#not taking into account the position of the driver?
for r in R:
    for d in D:
        for j in J:
            for k in mot:
                if pickup_id_job[j] == pickup_id[r]: # this should lead to many rows 'r' matching with 'j'
                    if dist_start_pickup[r] > gm_radius: #does this take all 'r' into account?
                        vars[d] = 0 #should I call them vars or drivers?
                    elif dist_start_pickup <= gm_radius:
                        if k == 'bike':
                            if (time_start_pickup[r], time_pickup_delivery[r]) > pickup_and_delivery_timeout_bike:
                                vars[d] = 0
                            elif (time_start_pickup[r], time_pickup_delivery[r]) <= pickup_and_delivery_timeout_bike:
                                vars[d] <= 1
                        elif k != 'bike':
                            vars[d] <= 1
                        
                elif pickup_id_job[j] != pickup_id[r]:
                    break

return total_time[r]
#print('total time is ' + total_time)                    

KeyError: 0

In [39]:
#create decision variable t = time
# we dont take MOT into account here? 
t = {}
for d in D:
    for j in J:
        t[d,j] = total_time[id[j]]

In [40]:
print(t[234,34])

1818.8


In [41]:
print(job_id[34])

d9d86a8af86876ec249bdf2b80dbc567


In [42]:
driver[234]

[(236,
  'Bicycle',
  '10/06/2019 17:00',
  '10/06/2019 22:10',
  1183436000.0,
  -0.070878999999999998,
  51.473205999999998)]

In [43]:
for j in J:
    print(pickup_id_job[34])
    break

2173488


In [44]:
#Decision Variable x[d,j] = 1 if driver d is assigned to job j

vars = {}
for d in D:
    for j in J:
        vars[d,j] = m.addVar(obj=total_time(i,j), vtype=GRB.BINARY, name = 'x')

TypeError: 'dict' object is not callable